In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
spam = pd.read_csv('../input/sms-spam-collection-dataset/spam.csv')
spam.head()

In [ ]:
spam.info()

In [ ]:
new = spam[['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']].copy()
na_string = '-'
spam['v2'] = spam['v2'].str.cat(new, sep = ", ", na_rep = na_string)
spam.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace = True)
spam

* I join all 3 unnamed columns into v2 because it's still contained with sentences.

# Text Pre-processing

### *Convert To Lower Case*

In [ ]:
def to_lower(text):
    return text.lower()

### *Expand Contraction*

In [ ]:
import re
contractions_dict = {     
"ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because",
"could've": "could have", "couldn't": "could not", "couldn't've": "could not have", "didn't": "did not", "doesn't": "does not",
"don't": "do not", "hadn't": "had not", "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not",
"he'd": "he had", "he'd've": "he would have", "he'll": "he will", "he'll've": "he will have", "he's": "he is",
"how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "I'd": "I had", "I'd've": "I would have",
"I'll": "I will", "I'll've": "I will have", "I'm": "I am", "I've": "I have", "isn't": "is not", "it'd": "it had",
"it'd've": "it would have", "it'll": "it will", "it'll've": "iit will have", "it's": "it is", "let's": "let us",
"ma'am": "madam", "mayn't": "may not", "might've": "might have", "mightn't": "might not", "mightn't've": "might not have",
"must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
"o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
"sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she had", "she'd've": "she would have", "she'll": "she will",
"she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not",
"shouldn't've": "should not have", "so've": "so have", "so's": "so is", "that'd": "that had", "that'd've": "that would have",
"that's": "that is", "there'd": "there had", "there'd've": "there would have", "there's": "there is", "they'd": "they had",
"they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are",
"they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we had", "we'd've": "we would have",
"we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not",
"what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have",
"when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have",
"who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is",
"why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have",
"wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
"y'all'd've": "you all would have", "y'all're": "you all are", "y'all've": "you all have", "you'd": "you had",
"you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"
}

def expand_contraction(text, contraction_dict):
    contraction_pattern= re.compile('({})'.format('|'.join(contraction_dict.keys())), flags= re.IGNORECASE | re.DOTALL)
    
    def expand_match(contraction):
        match= contraction.group(0)
        first_char= match[0]
        expanded_contraction= contraction_dict.get(match) \
            if contraction_dict.get(match) \
            else contraction_dict.get(match.lower())
        expanded_contraction= expanded_contraction
        return expanded_contraction
        
    expanded_text= contraction_pattern.sub(expand_match, text)
    expanded_text= re.sub("'","", expanded_text)
    return expanded_text

def main_contraction(text):
    text = expand_contraction(text, contractions_dict)
    return text

### *Remove or Convert Number*

In [ ]:
def remove_number(text):
    output = ''.join(c for c in text if not c.isdigit())
    return output

### *Remove Punctuation*

In [ ]:
from string import punctuation
def remove_punct(text):
    return "".join(c for c in text if c not in punctuation)

### *Remove WhiteSpaces*
* I also remove a word that only has one letter.

In [ ]:
def to_strip(text):
    return " ".join([c for c in text.split() if len(c)>1])

### *Remove Special Characters*
* This regex expression states that match the text string for any alphabets from 'small a' to 'small z or 'capital A' to 'capital Z'. I remove spaces again using the \s pattern, which refers to a single space.

In [ ]:
def remove_char(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)
    return text

### *Remove Duplicates*
* Removing multiple characters (duplicates) depends on the noise content in the dataset and it should be performed before removing stopwords. (.) match and capture any single character \1{2,} then match the same character two or more times. The quantity \1 represents the first capture group in sub.

In [ ]:
def remove_duplicate(text):
    text = re.sub("(.)\\1{2,}", "\\1", text)
    return text

### *Remove StopWords or Particular Words*

In [ ]:
import nltk
from nltk.corpus import stopwords
stopwords.words('english')

def remove_stopwords(text):
    stop_words= stopwords.words('english')
    
    return ' '.join(c for c in nltk.word_tokenize(text) if c not in stop_words)

### *Lemmatization*
* Lemmatization usually refers to doing proper vocabulary and morphological analysis of words, normally aiming to remove inflectional endings only and to return the base or dictionary form of a word. I prefer to use Lemmatization rather than Stemming in the text normalization process because Lemmatization considers the context and converts the word to its meaningful base form. Sometimes, the same word can have multiple different Lemma.

In [ ]:
from nltk.stem import WordNetLemmatizer

wordnet_lemma = WordNetLemmatizer()

def lemma(text):
    lemmatize_words = [wordnet_lemma.lemmatize(word) for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    return ' '.join(lemmatize_words)

### Result From Text Pre-processing 

In [ ]:
spam['prep1']= spam['v2'].apply(to_lower)
spam['prep2']= spam['prep1'].apply(main_contraction)
spam['prep3']= spam['prep2'].apply(remove_number)
spam['prep4']= spam['prep3'].apply(remove_punct)
spam['prep5']= spam['prep4'].apply(to_strip)
spam['prep6']= spam['prep5'].apply(remove_char)
spam['prep7']= spam['prep6'].apply(remove_duplicate)
spam['prep8']= spam['prep7'].apply(remove_stopwords)
spam['lemma'] = spam['prep8'].apply(lemma)
spam

# Text Exploration

### *Length Of Sentence*
* Identify if spam and ham text can differ in other content, such as length of text.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
spam['length'] = spam['v2'].apply(len)

plt.figure(figsize = (15, 6))
spam0 = spam[spam['v1'] == 'ham']
spam1 = spam[spam['v1'] == 'spam']
sns.distplot(spam0['length'])
sns.distplot(spam1['length'])
plt.legend(['Ham', 'Spam'])
plt.show()

In [ ]:
spam['length'] = spam['v2'].apply(len)

plt.figure(figsize = (15, 6))
spam2= spam[spam['length']<200]
spam0 = spam2[spam2['v1'] == 'ham']
spam1 = spam2[spam2['v1'] == 'spam']
sns.distplot(spam0['length'])
sns.distplot(spam1['length'])
plt.legend(['Ham', 'Spam'])
plt.show()

* I try to see the differences in the length of words in every class. From the plots, I can conclude that Spam and Ham almost have an equal length, but the Spam density is higher than the Ham.

### *Words Frequency*
* Can be used to check whether or not there are still words frequantly occur but not meaningful

In [ ]:
def dictionary(check):
    check = check.str.extractall('([a-zA_Z]+)')
    check.columns = ['check']
    b = check.reset_index(drop=True)
    check = b['check'].value_counts()
    
    dictionary = pd.DataFrame({'word': check.index, 'freq': check.values})
    dictionary.index = dictionary['word']
    dictionary.drop('word', axis = 1, inplace=True)
    dictionary.sort_values('freq', inplace= True, ascending= False)
    
    return dictionary

dictionary_clean = dictionary(spam['lemma'])
dictionary_clean[:20].plot(kind = 'barh',figsize = (10,10))

# Modeling

### *Data Splitting*

In [ ]:
from sklearn.model_selection import train_test_split

* The data is indicated for imbalanced but I decide not to process for further because the score is already higher.

In [ ]:
pd.DataFrame(spam['v1'].value_counts()/spam.shape[0]*100).round(2)

* *0 = Ham*
* *1 = Spam*

        - TN: SMS predict with Ham and the actual is Ham
        - TP: SMS predict with Spam and the actual is Spam
        - FP: SMS predict with Spam and the actual is Ham
        - FN: SMS predict with Ham and the actual is Spam

Actions:
* FN: The prediction is inaccurate. The spam SMS will be sent and the receiver will be annoyed because of that although it can be deleted manually.
* FP: If ham SMS is important, it may be removed as spam based on the prediction.

    -> Of all the results, the metric that I use to determine the score is using f1-score (f1) to anticipate error prediction.

In [ ]:
text = spam['lemma']
y = np.where(spam['v1'] == 'spam', 1, 0)

text.shape

In [ ]:
text_train, text_test, y_train, y_test = train_test_split(text, y, 
                                                          stratify = y, 
                                                          test_size = 0.3, 
                                                          random_state = 1672)

* I use 0.3 as test_size, which means 70% as train data and 30% as test data.

### *Define Model*
* Most term frequency is not a good measure of the importance of a word/term to a document's topic. Very common words like "the", "a", "to" is almost always the terms with the highest frequency in the text. To circumvent the elimination of term-frequency, I normalize it by the inverse document frequency (IDF). The result of that is the TF-IDF matrix. The inverse document frequency is a measurement of how much information the word provides, that is, whether the term is common or rare across all documents in the corpus.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, precision_score, accuracy_score, f1_score
from sklearn.pipeline import Pipeline

In [ ]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english', token_pattern=r'\w{1,}')
logreg = LogisticRegression(random_state = 1672)
bayes = MultinomialNB()
lsvc = LinearSVC(random_state = 1672)
rf = RandomForestClassifier(random_state = 1672)

logreg_pipe = Pipeline([('vector', tfidf), ('model', logreg)])
bayes_pipe = Pipeline([('vector', tfidf), ('model', bayes)])
lsvc_pipe = Pipeline([('vector', tfidf), ('model', lsvc)])
rf_pipe = Pipeline([('vector', tfidf), ('model', rf)])

def model_evaluation(model, metric):
    model_cv = cross_val_score(model, text_train, y_train, cv = StratifiedKFold(n_splits = 5), scoring = metric)
    return model_cv

logreg_pipe_cv = model_evaluation(logreg_pipe, 'f1')
bayes_pipe_cv = model_evaluation(bayes_pipe, 'f1')
lsvc_pipe_cv = model_evaluation(lsvc_pipe, 'f1')
rf_pipe_cv = model_evaluation(rf_pipe, 'f1')

score_cv = [logreg_pipe_cv.round(5), bayes_pipe_cv.round(5), lsvc_pipe_cv.round(5), rf_pipe_cv.round(5)]
score_mean = [logreg_pipe_cv.mean(), bayes_pipe_cv.mean(), lsvc_pipe_cv.mean(), rf_pipe_cv.mean()]
score_std = [logreg_pipe_cv.std(), bayes_pipe_cv.std(), lsvc_pipe_cv.std(), rf_pipe_cv.std()]

for model in [logreg_pipe, bayes_pipe, lsvc_pipe, rf_pipe]:
    model.fit(text_train, y_train)

score_f1 = [f1_score(y_test, logreg_pipe.predict(text_test)),
                  f1_score(y_test, bayes_pipe.predict(text_test)),
                  f1_score(y_test, lsvc_pipe.predict(text_test)),
                  f1_score(y_test, rf_pipe.predict(text_test))]

method_name = ['Logistic Regression Tfidf Vectorizer', 'Multinomial Naive Bayes Tfidf Vectorizer',
               'Linear SVC Tfidf Vectorizer', 'Random Forest Classifier Tfidf Vectorizer']

summary = pd.DataFrame({'method': method_name,
                            'cv score': score_cv,
                            'mean score': score_mean,
                            'std score': score_std,
                            'f1 score': score_f1})
summary

* This is the score that I will use to continue the process. Based on the Linear SVC model with TF-IDF Vectorizer, it has the highest mean score, the std score is the lowest one meaning it's good, and also has the highest f1 score. Let's continue to see how the classification report supports this score.

In [ ]:
for model, model_name in zip([logreg_pipe, bayes_pipe, lsvc_pipe, rf_pipe],
                             ['Logistic Regression Tfidf Vectorizer', 'Multinomial Naive Bayes Tfidf Vectorizer',
                              'Linear SVC Tfidf Vectorizer', 'Random Forest Classifier Tfidf Vectorizer']):
    model.fit(text_train, y_train)
    y_pred = model.predict(text_test)
    print(model_name+ ':')
    print(classification_report(y_test, y_pred))

* The highest f1 score based on the classification report is the Linear SVM model with TF-IDF Vectorizer.

# HyperParameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

* This is one way to improve the f1 score with tuning using Grid Search. I choose this library because Grid Search will give the best improvement score. In fact, Grid Search is heavy to run because it's working with the parameters one by one for each data.

In [ ]:
lsvc_estimator = Pipeline([('vector', tfidf), ('model', lsvc)])

hyperparam_space = {
    'vector__analyzer': ['word', 'char','char_wb'],
    'vector__max_features': [2000, 3000, 5000],
    'model__C': [0.01, 0.1, 1, 10],
    'model__multi_class': ['ovr', 'crammer_singer'],
    'model__class_weight': ['dict', 'balanced'],
    'model__random_state': [1672]
}

grid = GridSearchCV(
                lsvc_estimator,
                param_grid = hyperparam_space,
                cv = StratifiedKFold(n_splits = 5),
                scoring = 'f1',
                n_jobs = -1)

grid.fit(text_train, y_train)

print('best score', grid.best_score_)
print('best param', grid.best_params_)

* After the tuning process, the f1 score is decreasing.

# Result Comparison

In [ ]:
lsvc_pipe.fit(text_train, y_train)
f1_lsvc = (f1_score(y_test, lsvc_pipe.predict(text_test)))

grid.best_estimator_.fit(text_train, y_train)
f1_grid = (f1_score(y_test, grid.predict(text_test)))

score_list = [f1_lsvc, f1_grid]
method_name = ['Linear SVC Tfidf Vectorizer Before Tuning',
               'Linear SVC Tfidf Vectorizer After Tuning']
best_summary = pd.DataFrame({
    'method': method_name,
    'score': score_list
})
best_summary

In [ ]:
for model, model_name in zip([lsvc_pipe, grid.best_estimator_],
                             ['Linear SVC Tfidf Vectorizer Before Tuning',
                              'Linear SVC Tfidf Vectorizer After Tuning']):
    model.fit(text_train, y_train)
    y_pred = model.predict(text_test)
    print(model_name+ ':')
    print(classification_report(y_test, y_pred))

* The score is almost equal between before and after tuning. Still, the highest f1 score is the Linear SVC model with TF-IDF Vectorizer before Tuning.

# Prediction Testing

In [ ]:
sms = ["Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005.",
       "Too late. I said i have the website. I didn't i have or dont have the slippers",
       "Sir, i am waiting for your call, once free please call me.",
       "how are you? I miss you!"]

In [ ]:
predictions = lsvc_pipe.predict(sms)
result = np.where(predictions == 1, 'Spam', 'Ham')
result